<a href="https://colab.research.google.com/github/JITHIN-ANTONY-JOSEPH/ERP_11358080/blob/main/9_Experiment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Input : Recipe Instructions
### Model : Word2Vec(Text) + GraphSAGE(Graph) + Neural Networks(Model)

### Mounting to connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Installing required libraries

In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.6 MB/s eta 0:00:00


### Importing required libraries , loading datasets and pre-processing

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
flavorgraph_df = pd.read_csv('/content/drive/My Drive/ERP/Dataset/nodes_191120.csv')  # Adjust the path as needed , this is the path to my personal Google Drive

In [ ]:
import pandas as pd
import json
import re
import jellyfish
from gensim.models import Word2Vec
from concurrent.futures import ProcessPoolExecutor

# Load the main dataset
with open('/content/drive/My Drive/ERP/modified_Processed_Layer1.json', 'r') as file: # Adjust the path as needed , this is the path to my personal Google Drive
    recipe1m_data = [json.loads(line) for line in file]

recipe1m_df = pd.DataFrame(recipe1m_data)

# Load the substitution pairs
substitution_pairs_df = pd.read_csv('/content/drive/My Drive/ERP/Recipe1MSubs_full.csv') # Adjust the path as needed , this is the path to my personal Google Drive

# Merge the datasets based on recipe_id (substitution_pairs_df) nd id (recipe1m_df)
merged_df = pd.merge(recipe1m_df, substitution_pairs_df, left_on= 'id', right_on='recipe_id')

# Example ingredient list for NER-like extraction (replace with your own comprehensive list or use NER model)
ingredient_list = set(flavorgraph_df[flavorgraph_df['node_type'] == 'ingredient']['name'].dropna().unique())

# Function to extract ingredients from instructions
def extract_ingredients_from_instructions(instructions, ingredient_list):
    extracted_ingredients = []
    for instruction in instructions:
        words = instruction.split()
        for word in words:
            if word in ingredient_list:
                extracted_ingredients.append(word)
    return extracted_ingredients

# Apply the extraction function
recipe1m_df['extracted_ingredients'] = recipe1m_df['processed_instructions'].apply(
    lambda instructions: extract_ingredients_from_instructions(instructions, ingredient_list) if isinstance(instructions, list) else []
)

# Prepare sentences for training
sentences = recipe1m_df['extracted_ingredients'].tolist()

# Add substitution contexts to sentences
for _, row in substitution_pairs_df.iterrows():
    ingredient1 = row['ingredient1']
    ingredient2 = row['ingredient2']
    sentences.append([ingredient1, ingredient2])

# Train the Word2Vec model
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=8)  # Increase 'workers' to utilize more CPU cores

### Defining embeddings

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import jellyfish
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
import networkx as nx
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv

# Load the knowledge graph
flavor_graph = nx.read_graphml('/content/drive/My Drive/ERP/knowledge_graph.graphml')

# Filter the graph to include only ingredient nodes
ingredient_nodes = [n for n, attr in flavor_graph.nodes(data=True) if attr['node_type'] == 'ingredient']
flavor_graph = flavor_graph.subgraph(ingredient_nodes)

# Create a mapping from node names to numerical indices
node_to_index = {node: i for i, node in enumerate(flavor_graph.nodes())}
index_to_node = {i: node for node, i in node_to_index.items()}

# Generate text-based embeddings using Word2Vec
# Assuming 'sentences' is already prepared from your text data
text_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=8)
text_embeddings = text_model.wv

node_features = np.array([text_embeddings[node] if node in text_embeddings else np.zeros(100) for node in flavor_graph.nodes()])
edge_index = np.array([(node_to_index[u], node_to_index[v]) for u, v in flavor_graph.edges()]).T

# Convert to PyTorch tensors
x = torch.tensor(node_features, dtype=torch.float32)
edge_index = torch.tensor(edge_index, dtype=torch.long)

# Define GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GraphSAGE, self).__init__()
        self.sage1 = SAGEConv(in_channels, out_channels)
        self.sage2 = SAGEConv(out_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.sage1(x, edge_index)
        x = torch.relu(x)
        x = self.sage2(x, edge_index)
        return x

# Initialize the GraphSAGE model
graph_sage_model = GraphSAGE(in_channels=100, out_channels=100)

# Prepare data for training
data = Data(x=x, edge_index=edge_index)

# Train the GraphSAGE model
optimizer = torch.optim.Adam(graph_sage_model.parameters(), lr=0.01)
graph_sage_model.train()

for epoch in range(200):  # Adjust epochs as needed
    optimizer.zero_grad()
    out = graph_sage_model(data.x, data.edge_index)
    loss = F.mse_loss(out, data.x)  # Using feature reconstruction as a training objective
    loss.backward()
    optimizer.step()
    if epoch % 20 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# Get graph embeddings
graph_embeddings = out.detach().numpy()

# Combine text and graph embeddings
def get_combined_embedding(ingredient, text_embeddings, graph_embeddings):
    if ingredient in text_embeddings and ingredient in node_to_index:
        text_emb = text_embeddings[ingredient]
        graph_emb = graph_embeddings[node_to_index[ingredient]]
        return np.concatenate([text_emb, graph_emb])
    else:
        return np.zeros(200)  # Return zero vector if ingredient is not found in either embedding

combined_embeddings = {ingredient: get_combined_embedding(ingredient, text_embeddings, graph_embeddings) for ingredient in text_embeddings.index_to_key}


Epoch 0, Loss: 0.4375673234462738
Epoch 20, Loss: 0.1426171064376831
Epoch 40, Loss: 0.07879400998353958
Epoch 60, Loss: 0.05029160901904106
Epoch 80, Loss: 0.034899357706308365
Epoch 100, Loss: 0.029627220705151558
Epoch 120, Loss: 0.015158927999436855
Epoch 140, Loss: 0.012557406909763813
Epoch 160, Loss: 0.017621295526623726
Epoch 180, Loss: 0.006378908175975084


In [ ]:
# Combine embeddings for training data
train_data = []
train_labels = []

for _, row in substitution_pairs_df.iterrows():
    ing1 = row['ingredient1']
    ing2 = row['ingredient2']
    combined_embedding1 = get_combined_embedding(ing1, text_embeddings, graph_embeddings)
    combined_embedding2 = get_combined_embedding(ing2, text_embeddings, graph_embeddings)

    train_data.append(combined_embedding1)
    train_labels.append(combined_embedding2)

train_data = torch.tensor(train_data, dtype=torch.float32)
train_labels = torch.tensor(train_labels, dtype=torch.float32)


<ipython-input-16-b9e48eeabc86>:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  train_data = torch.tensor(train_data, dtype=torch.float32)


### Defining the neural network and combined model run

In [ ]:
# Define the neural network
class CombinedNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(CombinedNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, output_dim)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Instantiate the model
nn_model = CombinedNN(input_dim=200, output_dim=200)  # Combined embedding dimension is 200 (100 + 100)
criterion = nn.MSELoss()
optimizer = optim.Adam(nn_model.parameters(), lr=0.001)

# Training loop
for epoch in range(200):
    nn_model.train()
    optimizer.zero_grad()
    outputs = nn_model(train_data)
    loss = criterion(outputs, train_labels)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


Epoch 1, Loss: 1.4918891191482544
Epoch 2, Loss: 1.4565880298614502
Epoch 3, Loss: 1.426780343055725
Epoch 4, Loss: 1.4033043384552002
Epoch 5, Loss: 1.3855185508728027
Epoch 6, Loss: 1.3706163167953491
Epoch 7, Loss: 1.3576475381851196
Epoch 8, Loss: 1.3462826013565063
Epoch 9, Loss: 1.337214469909668
Epoch 10, Loss: 1.3289775848388672
Epoch 11, Loss: 1.3215599060058594
Epoch 12, Loss: 1.3148607015609741
Epoch 13, Loss: 1.3082935810089111
Epoch 14, Loss: 1.3017737865447998
Epoch 15, Loss: 1.2966320514678955
Epoch 16, Loss: 1.2914416790008545
Epoch 17, Loss: 1.2861120700836182
Epoch 18, Loss: 1.2811558246612549
Epoch 19, Loss: 1.2768816947937012
Epoch 20, Loss: 1.2723435163497925
Epoch 21, Loss: 1.2688885927200317
Epoch 22, Loss: 1.2654930353164673
Epoch 23, Loss: 1.2618346214294434
Epoch 24, Loss: 1.257851243019104
Epoch 25, Loss: 1.256516456604004
Epoch 26, Loss: 1.2526763677597046
Epoch 27, Loss: 1.2495944499969482
Epoch 28, Loss: 1.2468396425247192
Epoch 29, Loss: 1.244803071022033

### Evaluation

In [ ]:
# Evaluate on validation set
val_data = []
val_labels = []

for idx, row in enumerate(substitution_pairs_df.itertuples()):
    if idx >= 500:  # Change to the first 500 entries
        break
    ing1 = row.ingredient1
    combined_embedding = get_combined_embedding(ing1, text_embeddings, graph_embeddings)

    val_data.append(combined_embedding)
    val_labels.append(row.ingredient2)

val_data = torch.tensor(val_data, dtype=torch.float32)
val_predictions = nn_model(val_data).detach().numpy()

# Function to find the top N most similar ingredients based on cosine similarity
def find_top_similar_ingredients(predicted_embedding, combined_embeddings, top_n=10):
    similarities = {}
    for ingredient, embedding in combined_embeddings.items():
        similarity = cosine_similarity(predicted_embedding.reshape(1, -1), embedding.reshape(1, -1))[0][0]
        similarities[ingredient] = similarity
    sorted_ingredients = sorted(similarities.items(), key=lambda item: item[1], reverse=True)
    return [ingredient for ingredient, similarity in sorted_ingredients[:top_n]]

# Function to calculate metrics with Jaro-Winkler similarity threshold
def calculate_metrics(predictions, ground_truths, combined_embeddings, top_n=10, threshold=0.8):
    mrr, hit_1, hit_3, hit_10 = 0.0, 0.0, 0.0, 0.0
    total = len(ground_truths)

    for pred, gt in zip(predictions, ground_truths):
        top_similar = find_top_similar_ingredients(pred, combined_embeddings, top_n=top_n)
        for rank, candidate in enumerate(top_similar, start=1):
            sim = jellyfish.jaro_winkler_similarity(gt, candidate)
            if sim >= threshold:
                mrr += 1.0 / rank
                if rank == 1:
                    hit_1 += 1.0
                if rank <= 3:
                    hit_3 += 1.0
                if rank <= 10:
                    hit_10 += 1.0
                break

    mrr /= total
    hit_1 /= total
    hit_3 /= total
    hit_10 /= total
    return mrr, hit_1, hit_3, hit_10

# Calculate metrics for the first 500 entries of the validation set
val_labels_str = val_labels[:500]  # Assuming labels are ingredient names
mrr, hit_1, hit_3, hit_10 = calculate_metrics(val_predictions, val_labels_str, combined_embeddings)

print(f"MRR: {mrr:.4f}, Hit@1: {hit_1:.4f}, Hit@3: {hit_3:.4f}, Hit@10: {hit_10:.4f}")


MRR: 0.1118, Hit@1: 0.0640, Hit@3: 0.1260, Hit@10: 0.2460
